In [1]:
import pandas as pd
import numpy as np
import json

# # 读取每个sheet的名字
# names_dep = {}
departments = [
    "采购",
    "测试",
    "品质",
    "业务",
    "生产",
    "技术",
    "行政",
    "财务",
    "JK办公室",
    "人力资源部",
]
path = './575K_5750101107-202511.xlsx'
month_n = path.split('-')[-1].split('.')[0][-2:]
# 读取字典
with open('names_dep.json', 'r', encoding='utf-8') as f:
    names_dep = json.load(f)
# departments = list(names_dep.keys())
df = pd.read_excel(path, sheet_name='账单明细', header=1)
df1 = df.copy()
# 提取关税费用
tax = 0
if (~df1[['序号', '到件地区']].iloc[-1].isnull()).all():
    tax = eval(df1['到件地区'].iloc[-1])
df2 = df.dropna().copy()
# 存储未分类的部门人员
unclassified = []


def dep_class(x):
    for dep in departments:
        if x in names_dep[dep]:
            return dep
    unclassified.append(x)


# 更改经手人名字
name_changes = {
    # '胡一炜': '陈伟',
    '如宝吴师傅': '陈伟',
    # '梁少锋': '裘高红',
    # '王方': '裘高红',
    # '小赖': '赖梦茜',
    # '章部长': '章燕罗',
    # '魏海泳': '蒋烨锴',
    # '吕经理': '吕委江',
    # '吕伟江': '吕委江',
    # '梁珮询': '梁珮珣',
    # '余作明': '俞作民',
    # '石': '石炯',
    # '代胜兰': '王方',
    # '潘小姐': '潘月',
    # '章生': '章毅',
    'Meisheng 紅/Susan': 'susan',
    # '小王': '王园漪',
    'Lia': '潘月',
    '贰贰': '潘孝秋',
}
df2['经手人'] = df2['经手人'].replace(name_changes)

df2['部门'] = df2['经手人'].apply(dep_class)
df2['部门'] = pd.Categorical(df2['部门'], categories=departments, ordered=True)

unclassified = sorted(set(unclassified))

if unclassified:
    for i in unclassified:
        print(i)
    print(f'\n共{len(unclassified)}人未分类')

# df2['经手人'] = df2['经手人'].str.replace('Meisheng Tia/Susan紅', 'Susan紅')
# 按照部门列的顺序排序


# 按照部门将df2分组，然后将每个部门的数据写入到不同的sheet中
with pd.ExcelWriter(f'{month_n}月_结果.xlsx') as writer:
    sum = df2.groupby('部门', observed=False)['费用(元)'].sum()
    discount = df2.groupby('部门', observed=False)['折扣/促销'].sum()
    summary_dataframe = sum.to_frame()
    summary_dataframe['折扣后金额明细'] = sum - discount
    summary_dataframe.reset_index(inplace=True)

    total_discount = discount.sum()
    summary_dataframe.rename(columns={'费用(元)': '金额'}, inplace=True)
    extra_rows = pd.DataFrame(
        {
            '部门': ['小计', '当月折扣', '关税费用', '总计'],
            '金额': [
                summary_dataframe['金额'].sum(),
                -total_discount,
                tax,
                summary_dataframe['金额'].sum() - total_discount + tax,
            ],
            '折扣后金额明细': [
                summary_dataframe['折扣后金额明细'].sum(),
                np.nan,
                np.nan,
                np.nan,
            ],
        },
        index=[0, 1, 2, 3],
    )
    print(extra_rows)

    summary_dataframe['备注'] = np.nan

    summary_dataframe = pd.concat([summary_dataframe, extra_rows])
    summary_dataframe.insert(0, '月份', f'{month_n}月')
    # print(summary_dataframe)
    summary_dataframe.to_excel(writer, sheet_name='汇总表', index=False)

    for dep in departments:
        # 序号列重新编号
        df2.loc[df2['部门'] == dep, '序号'] = range(
            1, df2['部门'].value_counts()[dep] + 1
        )
        # 写入到不同的sheet中
        df_dep = df2[df2['部门'] == dep].copy()
        # 将两人放在一起
        pri = {'王方': 1, '裘高红': 1}
        df_dep['weight'] = df_dep['经手人'].map(pri).fillna(3).astype(int)
        df_dep.sort_values(by=['weight', '经手人'], inplace=True)
        df_dep.drop(columns='weight', inplace=True)
        sum_row = [
            {
                '费用(元)': df_dep['费用(元)'].sum(),
                '应付金额': df_dep['应付金额'].sum(),
            }
        ]
        df_dep = pd.concat([df_dep, pd.DataFrame(sum_row)])

        df_dep.to_excel(writer, sheet_name=dep, index=False)


d:\application\anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


SHIRLEY YANG  楊英 
Vera
孙先生
孙杰宏
官厂
小苏
尹杰
汪巧芳
章丽紅
美盛控股集团有限公司

共10人未分类
     部门        金额   折扣后金额明细
0    小计  13716.00  13071.86
1  当月折扣   -644.14       NaN
2  关税费用      0.00       NaN
3    总计  13071.86       NaN


In [11]:
# 提取采购部门
pd.read_excel(f'{month_n}月_结果.xlsx', sheet_name='采购').to_excel(
    f'{month_n}月_采购.xlsx', index=False, sheet_name='采购'
)

In [ ]:
# 经手人中包含names_changes中的值的行
names_changes = {
    '如宝吴师傅': '陈伟',
    '梁少锋': '裘高红',
    '王方': '裘高红',
    # '小赖': '赖梦茜',
    # '章部长': '章燕罗',
    '魏海泳': '蒋烨锴',
    # '吕经理': '吕委江',
    # '吕伟江': '吕委江',
}
a = df2[df2['经手人'].isin(names_changes.keys())].copy()
b = df2[df2['经手人'].isin(names_changes.values())].copy()

s = (
    b.groupby(
        [
            '经手人',
            '寄件地区',
            '到件地区',
            '对方公司名称',
        ]
    )
    .size()
    .reset_index(name='counts')
)
print(a)
a = a.replace(
    {'经手人': names_changes},
)

idx = s.groupby('经手人')['counts'].idxmax()
print(idx)
max_company_names = s.iloc[idx, :-1]
t = pd.merge(a, max_company_names, on='经手人', how='left')
_x = [c for c in t.columns if '_x' in c]
t.drop(columns=_x, inplace=True)
# 去除_y
t.columns = [c.replace('_y', '') for c in t.columns]
t = t[a.columns]
t.index = a.index
print(t)

In [3]:
# 填写对应数字
rel = '444404'
unclassified = list(unclassified)
map_data = [
    {0: '采购', 1: '1'},
    {0: '测试', 1: '2'},
    {0: '品质', 1: '3'},
    {0: '业务', 1: '4'},
    {0: '生产', 1: '5'},
    {0: '技术', 1: '6'},
    {0: '行政', 1: '7'},
    {0: '财务', 1: '8'},
    {0: 'JK办公室', 1: '9'},
    {0: '无', 1: '0'},
    {0: '人力资源部', 1: 'a'},
]
unclassified_deps = []
for i in rel:
    for j in map_data:
        if i == j[1]:
            unclassified_deps.append(j[0])

pd.DataFrame({'人员': unclassified, '部门': unclassified_deps}).to_excel(
    './unclass.xlsx', index=False, header=True
)

In [4]:
# 向names_dep中添加新的人员
df = pd.read_excel('./unclass.xlsx')
dict1 = df[~(df['部门'] == '无')].groupby('部门')['人员'].apply(list).to_dict()
# 读取字典
with open('names_dep.json', 'r', encoding='utf-8') as f:
    names_dep = json.load(f)
# print(dict1)
for k, v in dict1.items():
    names_dep[k] += v
print(names_dep)
# 写入到json文件

with open('names_dep.json', 'w', encoding='utf-8') as f:
    json.dump(names_dep, f, ensure_ascii=False, indent=4)

{'采购': ['吕委江', '蒋烨锴', '裘高红', '吴松', '王园漪', '陈小姐', '丁霄逍', '俞作民', '陈伟', '官伟龙', '陈莎云', '吴淞', '杨朝伟', '小裘', '俞作', '小官', '章燕罗', '赖梦茜', '蒋烨楷', '潘茜茜', '俞作明', '丁宵逍', '丁而', '石天成', '徐晴晴', '王方', '袁杭斌', '石春英', '章生', '赵宝才', '阿夏'], '测试': ['Lily Liu', '俞铖钰', '吴志德', '王美华', '吕泽茜', 'VERA', 'Lily'], '品质': ['潘洁', '赵伟', '龚耀华', 'Sunny Zhu', 'Felicia/Sunny'], '业务': ['Shirley Yang', '张甜', '章毅', '董彬炳', '何伶俐', 'MeiShengFeliMaggie', '吕明敏', '小安', 'SHIRLEY YANG (楊英)', '潘音因', '孙薇', 'Hannah Shu', '梁珮珣', 'Meisheng Tia', '安怡', '潘音因 ', 'Linda', 'Susan', 'Zero.Lyu', '孫薇 Jenny Sun', 'Shally', '潘音因 Pany Pan', '韦丽', '盛议民', '王海东', '盛益铭', 'Meisheng Susan', '刘洁', 'karen', '章丽红', 'ShirleyYang', '裘亚丽', 'Felicia', 'hannah shu', 'Meisheng Tia/Susan', 'Shelley', '张伟莉', 'Tina', 'SHIRLEY YANG楊英', 'ANN', '陈可卿', 'Shirley 楊英', 'SHIRLEY YANG 楊英', '小胡', 'Linda Zhang', 'Mally', 'Susan.Lyu', '楊英', 'Meisheng Tia/Susan紅', 'Zero', '李加林', 'Meisheng Susan / 紅', '梁云云', '潘孝秋', 'Maggie', 'Aimee', '杨英', '李乐乐', 'Hong', '张萍', '王圆圆', 'Ann', 'Tia', 'magg

In [ ]:
import pandas as pd

df = pd.read_excel('./575K_5750101107-202411.xlsx', sheet_name='账单明细', header=1)
if (~df[['序号', '到件地区']].iloc[-1].isnull()).all():
    tax = eval(df['到件地区'].iloc[-1])
df['折扣/促销']